In [51]:
vocab = open('../04. Model of DP/precomputed/vocab.txt', 'r').read().splitlines()
info(f'len(vocab)={len(vocab)}')
np.array(vocab)

2022-04-01 21:38:26.046912 [ INFO  ] len(vocab)=10000


array(['title', 'new', 'toronto', ..., 'smoothly', 'roil', 'aril'],
      dtype='<U16')


### Corpus Small

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
from sklearn.neural_network import MLPClassifier

from utils.general import info, warning
import numpy as np
from utils.models import DataItem

## PARAMS
WITH_IDF=False
CUSTOM_VOCAB = False
BALANCED=True
MODEL = 'SVM'
SMALL=False
VECTOR_TYPE = DataItem.TYPE_BOW
##


vocabulary=None
if CUSTOM_VOCAB:
    vocabulary = open('../04. Model of DP/precomputed/vocab.txt', 'r').read().splitlines()

from utils.models import DataItem
corpus_small=[]
corpus_big=[]
#########################
# NOT MENTIONING CANADA #
#########################
IN_CANADA_FILE = './auxiliary_notebooks/in_canada.csv'
lines = open(IN_CANADA_FILE,'r').read().splitlines()
irrelevant_ids = set([line.split(';')[0] for line in lines if line.endswith('False')])
corpus_big = [DataItem(id_) for id_ in irrelevant_ids]
for item in corpus_big:
    item.set_irrelevant()

#############################
# LABELED OVER THREE ROUNDS #
#############################
for line in open('labeled_data.csv').read().splitlines()[1:]:
    id_,label = line.split(';')
    item = DataItem(id_)
    if label=='R':
        item.set_relevant()
    else:
        item.set_irrelevant()
        assert label=='I'
    if item.has_vector():
        corpus_small.append(item)
        corpus_big.append(item)
len(corpus_big)
        

assert len(corpus_small)==7104
assert len(corpus_big)==114492
info(f'SMALl: {len(corpus_small):10,}') 
info(f'BIG:   {len(corpus_big):10,}') 
    
small_balanced = []
big_balanced = []
#########
## BIG ##
#########
relevant = [item for item in corpus_big if item.label==DataItem.REL_LABEL]

rng = np.random.default_rng(2022)
irrelevants = rng.choice([item for item in corpus_big if item.label==DataItem.IREL_LABEL],
                         size=len(relevant),
                         replace=False
                        )
info(len(relevant))
info(len(irrelevants))

big_balanced = relevant+list(irrelevants)
rng.shuffle(big_balanced)
info(f'Using BALANCED, size of the corpus ({len(relevant):,} R + {len(irrelevants):,} I) = {len(big_balanced):,}')

###########
## SMALL ##
###########
relevant = [item for item in corpus_small if item.label==DataItem.REL_LABEL]


irrelevants = rng.choice([item for item in corpus_small if item.label==DataItem.IREL_LABEL],
                         size=len(relevant),
                         replace=False
                        )
info(len(relevant))
info(len(irrelevants))

small_balanced = relevant+list(irrelevants)
rng.shuffle(small_balanced)
info(f'Using BALANCED, size of the corpus ({len(relevant):,} R + {len(irrelevants):,} I) = {len(small_balanced):,}')


nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

stopwords = nlp.Defaults.stop_words
invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) 
stopwords = set(stopwords.difference(invalid)) 
info('using CUSTOM VOCAB.')
vectorizer = TfidfVectorizer(
                             tokenizer=tokenize,
                             stop_words=list(stopwords),
                             max_features=10000,
#                              vocabulary=None,
                             use_idf=False)
vectorizer.fit(['. '.join(list(TDMStudio.get_title_and_text(item.filename()))) 
                                   for item in  corpus_big ])

X_1 = vectorizer.transform(['. '.join(list(TDMStudio.get_title_and_text(item.filename()))) 
                                   for item in  corpus_small ])
X_2 = vectorizer.transform(['. '.join(list(TDMStudio.get_title_and_text(item.filename()))) 
                                   for item in  small_balanced ])

X_3 = vectorizer.transform(['. '.join(list(TDMStudio.get_title_and_text(item.filename()))) 
                                   for item in  big_balanced ])

info(f'X_1.shape={X_1.shape}')
info(f'X_2.shape={X_2.shape}')
info(f'X_3.shape={X_3.shape}')
# else:
#     info('Using UNBALANCED data')
    
# if VECTOR_TYPE!=DataItem.TYPE_BOW:
#     str_ = 'Using ' + ('GloVe300' if VECTOR_TYPE==DataItem.TYPE_GLOVE300 else 'GloVe600')
#     info(str_)
#     X = DataItem.get_X(corpus, type_=VECTOR_TYPE)
#     assert X.shape[1]==300 or X.shape[1]==600

# else:
#     str_ = 'Using BOW ' + ('with IDF' if WITH_IDF else 'WITHOUT IDF')
#     info(str_)
#     nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

#     stopwords = nlp.Defaults.stop_words
#     invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) 
#     stopwords = set(stopwords.difference(invalid)) 

#     if CUSTOM_VOCAB:
#         info('using CUSTOM VOCAB.')
#     vectorizer = TfidfVectorizer(
#                                  tokenizer=tokenize,
#                                  stop_words=list(stopwords),
#                                  max_features=10000,
#                                  vocabulary=vocabulary,
#                                  use_idf=WITH_IDF)

#     X = vectorizer.fit_transform(['. '.join(list(TDMStudio.get_title_and_text(item.filename()))) 
#                                        for item in  corpus ])
    
# y = DataItem.get_y(corpus)
# info(f'X.shape={X.shape}')
# info(f'y.shape={y.shape}')


# if MODEL=='SVM':
#     info('Using SVM MODEL')
#     params = {'kernel':['rbf','linear', 'poly'],  'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
#     model = SVC()
# elif MODEL=='LogisticRegression':
#     info('Using LOGISTiC REGRESSION MODEL')
#     params = {'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
#     model = LogisticRegression(max_iter=1500)
# elif MODEL=='MLPClassifier':
#     info('Using MULTI LAYER PERCEPTRON MODEL')
#     params = {'early_stopping': [True, False], 
#               'hidden_layer_sizes':[(50,),(100,), (100,10), (100,50), (100,20,10,5,3,2), (20,),(20,10,5,3,2)],
#               'solver':['adam', 'lbfgs']
#              }
#     model = MLPClassifier(max_iter=1500)

# GS = GridSearchCV(model,                      
#                   params,
#                   scoring='f1',
#                   cv=5,
#                   verbose=1, 
#                   n_jobs=4
# )    
# results = GS.fit(X,y)

# df = pd.DataFrame(results.cv_results_).sort_values(by='rank_test_score').head()

# print(df.loc[:,[column for column in df.columns if not 'param_' in column and not 'split' in column and not 'time' in column and not 'std' in column]])


# if MODEL=='SVM' and results.best_params_['kernel']=='poly':
#     warning('Best kernel found=poly, re-doing GridSearch to find best value for param "degree"')
#     del(params['kernel'])
#     params['degree'] = [1,2,3,5,7]
#     GS = GridSearchCV(SVC(kernel='poly'),                      
#                   params,
#                   scoring='f1',
#                   cv=5,
#                   verbose=1, 
#                   n_jobs=4
#     )    
#     results = GS.fit(X,y)
#     df = pd.DataFrame(results.cv_results_).sort_values(by='rank_test_score').head()

#     print(df.loc[:,[column for column in df.columns if not 'param_' in column and not 'split' in column and not 'time' in column and not 'std' in column]])

# best_model = results.best_estimator_
# info(f'Best model: {best_model}')
# info(f'Best params: {results.best_params_}')
# info(f'Best results: {results.best_score_}')
# info('Computing Cross Validation with best model')
# from sklearn.model_selection import cross_validate
# cv = cross_validate(
#                     best_model,
#                     X,
#                     y,
#                     scoring=['accuracy', 'precision','recall','f1']
#                     )
# df = pd.DataFrame(cv)
# import numpy as np
# pd.DataFrame(np.average(df.values,axis=0).reshape(1,6), columns=df.columns)

2022-04-05 20:35:13.438416 [ INFO  ] SMALl:      7,104
2022-04-05 20:35:13.438566 [ INFO  ] BIG:      114,492
2022-04-05 20:35:13.632896 [ INFO  ] 581
2022-04-05 20:35:13.633041 [ INFO  ] 581
2022-04-05 20:35:13.633278 [ INFO  ] Using BALANCED, size of the corpus (581 R + 581 I) = 1,162
2022-04-05 20:35:13.643007 [ INFO  ] 581
2022-04-05 20:35:13.643133 [ INFO  ] 581
2022-04-05 20:35:13.643342 [ INFO  ] Using BALANCED, size of the corpus (581 R + 581 I) = 1,162
2022-04-05 20:35:14.818633 [ INFO  ] using CUSTOM VOCAB.


In [21]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.20, random_state=42)

In [22]:
print(np.sum(y_train))
print(np.sum(y_test))

446.0
135.0


In [33]:
print(np.sum(y_train==0))
print(np.sum(y_test==0))

5237
1286


In [54]:
all_args = np.array(range(len(y_train)))
print(len(all_args))
print(all_args)
print()

rel_args = all_args[y_train==1]
print(len(rel_args))
print(rel_args[:20])
print()

irrel_args = all_args[y_train==0]
print(len(irrel_args))
print(irrel_args[:20])
print()

selected_irrel_args =np.random.choice(irrel_args, size=len(rel_args),replace=False)
print(len(selected_irrel_args))
print(selected_irrel_args[:20])
print()

print(X_train[np.hstack([rel_args,selected_irrel_args]),:].shape)
X_balanced = X_train[np.hstack([rel_args,selected_irrel_args]),:]
y_balanced = y_train[np.hstack([rel_args,selected_irrel_args])]
# print(np.sum(rel_args))
# print(np.sum(irrel_args))
# print(np.sum(selected_irrel_args))

5683
[   0    1    2 ... 5680 5681 5682]

446
[  0  13  19  40  42  70  84  89  91 106 108 114 136 155 156 168 177 216
 227 236]

5237
[ 1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 20 21 22]

446
[5650 1755  974 4623  315 1850 5480 3441 3296  664 2475 1749 2338 3914
 3663 5606 1533  721 2923 3849]

(892, 10000)


In [50]:
np.hstack([rel_args,selected_irrel_args])

(892,)

In [56]:
model1 = SVC(kernel='linear',C=1)
model2 = SVC(kernel='linear',C=1, class_weight='balanced')
model3 = SVC(kernel='linear',C=1)


model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_balanced,y_balanced)

SVC(C=1, kernel='linear')

In [57]:
from sklearn.metrics import f1_score
for idx, model in enumerate([model1,model2,model3]):
#     ytrue = [y_1,y_2,y_3][idx]
#     x = [X_1,X_2,X_3][idx]
    print(f1_score(y_test, model.predict(X_test)))

0.8872180451127819
0.8686868686868687
0.7584269662921349


In [31]:

args = np.random.choice(range(len, size=None, replace=True, p=None

SyntaxError: unexpected EOF while parsing (1302026171.py, line 1)

In [ ]:
args = np.random.choice

In [ ]:
args = np.choice

In [12]:
for x in [X_1, X_2, X_3]:
    print(x.shape)

(7104, 10000)
(1162, 10000)
(1162, 10000)


In [13]:

y_1 = DataItem.get_y(corpus_small)
y_2 = DataItem.get_y(small_balanced)
y_3 = DataItem.get_y(big_balanced)



In [18]:
model1 = SVC(kernel='linear', C=1)
model2 = SVC(kernel='linear', C=1)
model3 = SVC(kernel='linear', C=1)
model4 = SVC(kernel='linear', C=1, class_weight='balanced')

model1.fit(X_1,y_1)
model2.fit(X_2,y_2)
model3.fit(X_3,y_3)
model4.fit(X_1,y_1)

SVC(C=1, class_weight='balanced', kernel='linear')

In [19]:
from sklearn.metrics import f1_score
for idx, model in enumerate([model1,model2,model3,model4]):
#     ytrue = [y_1,y_2,y_3][idx]
#     x = [X_1,X_2,X_3][idx]
    print(f1_score(y_1, model.predict(X_1)))

0.9387402933563416
0.7443946188340806
0.6609783845278725
0.916403785488959


In [ ]:
from sklearn.metrics import f1_score

yhat1 = best_model.predict(X)

f1_score(yhat1,y)



In [25]:
results.best_params_['kernel']

'linear'

In [23]:
df = pd.DataFrame(results.cv_results_).sort_values(by='rank_test_score').head()

print(df.loc[:,[column for column in df.columns if not 'param_' in column and not 'split' in column and not 'time' in column and not 'std' in column]])

                          params  mean_test_score  rank_test_score
13  {'C': 7, 'kernel': 'linear'}         0.963936                1
14    {'C': 7, 'kernel': 'poly'}         0.963891                2
17   {'C': 12, 'kernel': 'poly'}         0.963782                3
11    {'C': 3, 'kernel': 'poly'}         0.963075                4
20   {'C': 15, 'kernel': 'poly'}         0.962085                5


### X, y bulding


In [29]:
X_small = DataItem.get_X(corpus_small, type_=DataItem.TYPE_GLOVE300)
y_small = DataItem.get_y(corpus_small)

### Grid Search

In [18]:

# params = {'kernel':['rbf','linear', 'poly'],  'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
params = {'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
# params = {  'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500],  'degree':[1,2,3,5,7]}

GS = GridSearchCV(
#                   SVC(kernel='poly'),
                  LogisticRegression(max_iter=1000),      
#                   SVC(),                         
                  params,
                  scoring='f1',
                  cv=5,
                  verbose=2, 
                  n_jobs=4
)    
results = GS.fit(X,y)

pd.DataFrame(results.cv_results_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.813785,0.030559,0.007606,0.000379,0.1,{'C': 0.1},0.435897,0.480519,0.447368,0.457516,0.484076,0.461076,0.018663,10
1,1.035526,0.077643,0.008034,0.000099,0.5,{'C': 0.5},0.764151,0.747475,0.832536,0.753769,0.813084,0.782203,0.034139,9
2,1.377653,0.050842,0.008098,0.000108,1,{'C': 1},0.803653,0.752475,0.846512,0.765550,0.837838,0.801206,0.037541,8
3,1.677906,0.129750,0.007926,0.000222,3,{'C': 3},0.821429,0.776699,0.850679,0.787037,0.850877,0.817344,0.031058,6
4,2.145715,0.206297,0.007817,0.000168,7,{'C': 7},0.842553,0.801887,0.844037,0.801802,0.854626,0.828981,0.022545,2
5,2.366602,0.155694,0.008000,0.000154,12,{'C': 12},0.835443,0.818605,0.844037,0.798206,0.847162,0.828690,0.018184,3
6,2.540038,0.145376,0.007877,0.000104,15,{'C': 15},0.840336,0.818605,0.844037,0.798206,0.847162,0.829669,0.018647,1
7,2.941976,0.237055,0.008180,0.000389,50,{'C': 50},0.841667,0.821918,0.828829,0.785714,0.847162,0.825058,0.021617,4
8,4.006620,0.135470,0.007865,0.000152,100,{'C': 100},0.840336,0.814480,0.828829,0.778281,0.847162,0.821817,0.024428,5
9,4.203352,0.348798,0.006448,0.001205,500,{'C': 500},0.823529,0.814480,0.816143,0.780269,0.842105,0.815305,0.020074,7


### Final Result

In [19]:
from sklearn.model_selection import cross_validate
cv = cross_validate(
                    LogisticRegression( C=15, max_iter=1000),
                    X,
                    y,
                    scoring=['accuracy', 'precision','recall','f1']
                    )
df = pd.DataFrame(cv)
import numpy as np
pd.DataFrame(np.average(df.values,axis=0).reshape(1,6), columns=df.columns)

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,1.740555,0.00823,0.973114,0.861496,0.801975,0.829669


# MISC

In [20]:
df

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,0.468034,0.199387,0.971851,0.818182,0.846154,0.831933
1,0.477119,0.197890,0.972555,0.846847,0.810345,0.828194
2,0.491669,0.203882,0.971851,0.827586,0.827586,0.827586
3,0.476155,0.199148,0.969740,0.823009,0.801724,0.812227
4,0.491318,0.197232,0.971127,0.809917,0.844828,0.827004


,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,0.480859,0.199508,0.971425,0.825108,0.826127,0.825389


In [9]:
import pandas as pd
pd.DataFrame(results.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
0,0.784086,0.015197,0.362105,0.006263,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.752852,0.781250,0.807843,0.732283,0.806084,0.776062,0.029643,28,0.954258,0.960591,0.965517,0.952146,0.964085,0.959319,0.005288,29,0.678082,0.714286,0.741007,0.673913,0.721088,0.705675,0.025808,30,0.846154,0.862069,0.887931,0.801724,0.913793,0.862334,0.038049,1
1,0.510647,0.009065,0.120951,0.002203,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.795276,0.805085,0.859504,0.722689,0.831373,0.802785,0.045851,11,0.963406,0.967628,0.976073,0.953554,0.969718,0.966076,0.007477,18,0.737226,0.791667,0.825397,0.704918,0.762590,0.764360,0.041818,28,0.863248,0.818966,0.896552,0.741379,0.913793,0.846788,0.061842,2
2,0.508384,0.010537,0.114645,0.002808,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.776371,0.764444,0.849785,0.708520,0.818182,0.783461,0.048234,26,0.962702,0.962702,0.975369,0.954258,0.969014,0.964809,0.007063,25,0.766667,0.788991,0.846154,0.738318,0.785714,0.785169,0.035416,22,0.786325,0.741379,0.853448,0.681034,0.853448,0.783127,0.066432,25
3,0.626605,0.016586,0.277074,0.006746,0.5,rbf,"{'C': 0.5, 'kernel': 'rbf'}",0.814516,0.791304,0.852321,0.733624,0.851406,0.808634,0.044044,7,0.967628,0.966221,0.975369,0.957072,0.973944,0.968047,0.006517,9,0.770992,0.798246,0.834711,0.743363,0.796992,0.788861,0.030482,21,0.863248,0.784483,0.870690,0.724138,0.913793,0.831270,0.067901,4
4,0.430266,0.009229,0.095875,0.002665,0.5,linear,"{'C': 0.5, 'kernel': 'linear'}",0.827869,0.791304,0.864407,0.720721,0.824490,0.805758,0.048414,9,0.970443,0.966221,0.977481,0.956369,0.969718,0.968046,0.006888,10,0.795276,0.798246,0.850000,0.754717,0.782946,0.796237,0.030971,13,0.863248,0.784483,0.879310,0.689655,0.870690,0.817477,0.072356,9
5,0.451240,0.010371,0.095023,0.003672,0.5,poly,"{'C': 0.5, 'kernel': 'poly'}",0.801688,0.769231,0.853448,0.735426,0.782979,0.788554,0.039015,23,0.966925,0.964110,0.976073,0.958480,0.964085,0.965934,0.005764,22,0.791667,0.809524,0.853448,0.766355,0.773109,0.798821,0.031193,11,0.811966,0.732759,0.853448,0.706897,0.793103,0.779635,0.053223,27
6,0.594478,0.020757,0.262382,0.004800,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.819672,0.796537,0.832618,0.759825,0.838710,0.809472,0.028729,6,0.969036,0.966925,0.972555,0.961295,0.971831,0.968328,0.004051,7,0.787402,0.800000,0.829060,0.769912,0.787879,0.794850,0.019610,16,0.854701,0.793103,0.836207,0.750000,0.896552,0.826113,0.050524,6
7,0.417184,0.010845,0.085899,0.002331,1,linear,"{'C': 1, 'kernel': 'linear'}",0.818182,0.800000,0.846154,0.723214,0.819672,0.801444,0.041793,12,0.969036,0.967628,0.974666,0.956369,0.969014,0.967343,0.005997,11,0.792000,0.807018,0.838983,0.750000,0.781250,0.793850,0.029310,17,0.846154,0.793103,0.853448,0.698276,0.862069,0.810610,0.061110,10
8,0.436608,0.010229,0.088184,0.002736,1,poly,"{'C': 1, 'kernel': 'poly'}",0.811966,0.800000,0.841202,0.745455,0.773913,0.794507,0.032698,17,0.969036,0.969036,0.973962,0.960591,0.963380,0.967201,0.004706,12,0.811966,0.846154,0.837607,0.788462,0.780702,0.812978,0.025888,6,0.811966,0.758621,0.844828,0.706897,0.767241,0.777910,0.047258,28
9,0.525494,0.010614,0.224670,0.001897,3,rbf,"{'C': 3, 'kernel': 'rbf'}",0.830645,0.815451,0.825532,0.761062,0.844262,0.815390,0.028708,4,0.970443,0.969740,0.971147,0.961999,0.973239,0.969314,0.003840,4,0.786260,0.811966,0.815126,0.781818,0.804688,0.799971,0.013515

### Corpus Big

In [3]:
from utils.models import DataItem
#########################
# NOT MENTIONING CANADA #
#########################
IN_CANADA_FILE = './auxiliary_notebooks/in_canada.csv'
lines = open(IN_CANADA_FILE,'r').read().splitlines()
irrelevant_ids = set([line.split(';')[0] for line in lines if line.endswith('False')])
corpus_big = [DataItem(id_) for id_ in irrelevant_ids]
for item in corpus_big:
    item.set_irrelevant()

#############################
# LABELED OVER THREE ROUNDS #
#############################
for line in open('labeled_data.csv').read().splitlines()[1:]:
    id_,label = line.split(';')
    item = DataItem(id_)
    if label=='R':
        item.set_relevant()
    else:
        item.set_irrelevant()
        assert label=='I'
    if item.has_vector():
        corpus_big.append(item)
len(corpus_big)

114492

In [4]:
X = DataItem.get_X(corpus_big, type_=DataItem.TYPE_GLOVE600)
y = DataItem.get_y(corpus_big)

In [5]:
print(X.shape)
print(y.shape)

(114492, 600)
(114492,)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from utils.general import info
import pandas as pd
params = {'kernel':['rbf','linear', 'poly'],  'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
# params = {'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500]}
# params = {  'C':[0.1, 0.5, 1 , 3, 7, 12, 15, 50, 100, 500],  'degree':[1,2,3,5,7]}
info('starting')
GS = GridSearchCV(
#                   SVC(kernel='poly'),
#                   LogisticRegression(max_iter=1000),      
                  SVC(),                         
                  params,
                  scoring='f1',
                  cv=5,
                  verbose=4, 
                  n_jobs=4
)    
results = GS.fit(X,y)

pd.DataFrame(results.cv_results_)

2022-04-01 16:55:12.892663 [ INFO  ] starting
Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 

In [32]:
from sklearn.model_selection import cross_validate
cv = cross_validate(
                    SVC( C=100,kernel='poly'),
#                     LogisticRegression( C=100,max_iter=1000),
                    X,
                    y,
                    scoring=['accuracy', 'precision','recall','f1']
                    )
df = pd.DataFrame(cv)
import numpy as np
pd.DataFrame(np.average(df.values,axis=0).reshape(1,6), columns=df.columns)

[CV] END ..............................................C=0.1; total time=   4.4s
[CV] END ..............................................C=0.5; total time=   8.3s
[CV] END ................................................C=1; total time=   9.5s
[CV] END ................................................C=1; total time=   7.0s
[CV] END ................................................C=3; total time=  13.6s
[CV] END ................................................C=7; total time=  14.5s
[CV] END ...............................................C=12; total time=  23.5s
[CV] END ...............................................C=15; total time=  24.9s
[CV] END ...............................................C=15; total time=  16.5s
[CV] END ...............................................C=50; total time=  30.4s
[CV] END ..............................................C=100; total time=  44.8s
[CV] END ..............................................C=500; total time= 1.1min
[CV] END ...................

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,26.871438,0.063222,0.995921,0.82209,0.74191,0.726835


In [27]:
X.shape

(114492, 300)

[CV] END ...............................C=0.1, kernel=linear; total time=  58.8s
[CV] END .................................C=0.1, kernel=poly; total time= 1.0min
[CV] END .................................C=0.1, kernel=poly; total time=  58.7s
[CV] END ..................................C=0.5, kernel=rbf; total time= 1.0min
[CV] END ...............................C=0.5, kernel=linear; total time=  55.8s
[CV] END .................................C=0.5, kernel=poly; total time=  57.2s
[CV] END ....................................C=1, kernel=rbf; total time=  57.6s
[CV] END .................................C=1, kernel=linear; total time=  53.4s
[CV] END .................................C=1, kernel=linear; total time=  37.2s
[CV] END ...................................C=1, kernel=poly; total time=  52.8s
[CV] END ....................................C=3, kernel=rbf; total time=  49.4s
[CV] END .................................C=3, kernel=linear; total time=  51.6s
[CV] END ...................

[CV] END ..............................................C=0.1; total time=   0.8s
[CV] END ..............................................C=0.5; total time=   1.0s
[CV] END ..............................................C=0.5; total time=   1.0s
[CV] END ................................................C=1; total time=   1.4s
[CV] END ................................................C=3; total time=   1.9s
[CV] END ................................................C=7; total time=   2.1s
[CV] END ...............................................C=12; total time=   2.3s
[CV] END ...............................................C=12; total time=   2.6s
[CV] END ...............................................C=15; total time=   2.4s
[CV] END ...............................................C=50; total time=   2.6s
[CV] END ..............................................C=100; total time=   3.8s
[CV] END ..............................................C=100; total time=   4.0s
[CV] END ...................

In [ ]:
clf = SVC(kernel='rbf', C=50)
clf.fit(X,y)